In [ ]:
#HOMEWORK

In [ ]:
#Question 1
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
!pip install requests

In [ ]:
# Question 2
##ADDING documents to Elastic Search

from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

In [ ]:
#Question 3

user_question = "How do I execute a command in a running docker container?"
es = Elasticsearch("http://localhost:9200")

# ^3 indicates that if user question matches the question in doc, it is given more priority when it matches with text or section
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text", "section"], 
                    "type": "best_fields"
                }
            },
            "filter": { 
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [ ]:
response = es.search(index=index_name, body=search_query)


In [ ]:
response['hits'].keys()

dict_keys(['total', 'max_score', 'hits'])

In [ ]:
response['hits']

{'total': {'value': 407, 'relation': 'eq'},
 'max_score': 75.54128,
 'hits': [{'_index': 'course-questions',
   '_id': '8f7fUpABu1U5aAdqyGKa',
   '_score': 75.54128,
   '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+-------------

In [ ]:
documents = [hit['_source'] for hit in response['hits']['hits']]
documents

[{'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
  'section': 'Module 1: Docker and Terraform',
  'question': 'PGCLI - running in a Docker container',
 

In [ ]:
#Question 4

#Question 3

user_question = "How do I execute a command in a running docker container?"

# ^3 indicates that if user question matches the question in doc, it is given more priority when it matches with text or section
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text", "section"], 
                    "type": "best_fields"
                }
            },
            "filter": { 
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es.search(index=index_name, body=search_query)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7a8f22b14ee0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [ ]:
documents = [hit['_source'] for hit in response['hits']['hits']]
for doc in documents:
    print(doc['question'])

TypeError: 'ChatCompletion' object is not subscriptable

In [ ]:
#Question 5
context_template = """
Q: {question}
A: {text}
""".strip()

In [ ]:

context = ""

for doc in documents:
    doc_str = f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    context += doc_str

context = context.strip()
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [ ]:
user_question = "How do I execute a command in a running docker container?"

prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {user_question}

CONTEXT:
{context}
""".strip()

In [ ]:
len(prompt_template)

1462

In [ ]:
#Question 6
# !pip install tiktoken
import tiktoken


In [ ]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0125")
encoded_tokens = encoding.encode(prompt_template)
len(encoded_tokens)

325

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")
encoded_tokens = encoding.encode(prompt_template)
len(encoded_tokens)

322

In [ ]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt_template}]
)
answer = response.choices[0].message.content
answer

'To execute a command in a running docker container, you can use the following command:\ndocker exec -it <container-id> bash'